In [2]:
import pandas as pd
import requests


In [3]:
df = pd.read_excel("/Users/danishluqman/Documents/horse-racing-arima-kinen-2013/data/results.xlsx")

In [4]:
col = df.pop("Horse")
df.insert(0, "Horse", col)

In [5]:
print(df.columns)

Index(['Horse', 'Race', 'Video', 'Fav', 'Fin.', 'J', 'Wgt.', 'Odds', 'Rnrs',
       'Bk', 'Draw', 'Dis.', 'WX', 'Gng.', 'FT', 'Mrg.', 'Pos. at Bends',
       'F3F-L3F', 'L3F', 'Horse Wgt.', 'Winner (Runner-up)', 'Prize (¥ mil)'],
      dtype='object')


In [6]:
df = df.drop(["Video", "Odds", "Fav", "Winner (Runner-up)", "Prize (¥ mil)", "WX"], axis = 1)

In [7]:
df.rename(columns={
    "J": "Jockey",
    "Fin.": "Finish",
    "Wgt.": "Weight",
    "Rnrs": "Runners",
    "Bk": "Block",
    "Gng.": "Ground",
    "Mrg.": "Margin",
    "Horse Wgt.": "Horse Weight"
}, inplace = True)
    
 

In [8]:
def time_to_seconds(time_str):
    if pd.isna(time_str):
        return None
    try:
        parts = time_str.split(":")
        if len(parts) == 2:
            minutes = int(parts[0])
            seconds = float(parts[1])
            return minutes*60 + seconds
        else:
            return float(parts[0])
    except:
        return None

df["FT_seconds"] = df["FT"].apply(time_to_seconds)
print(df["FT_seconds"].head(5))
            
            
    

0    152.30
1       NaN
2    161.47
3    119.00
4    143.10
Name: FT_seconds, dtype: float64


In [9]:
df["Track"] = df["Dis."].str[0].map({"T": "Turf", "D": "Dirt"})
df["Distance"] = df["Dis."].str[1:].astype(int)
df.drop("Dis.", axis = 1)

,Horse,Race,Finish,Jockey,Weight,Runners,Block,Draw,Ground,FT,Margin,Pos. at Bends,F3F-L3F,L3F,Horse Weight,FT_seconds,Track,Distance
0,Orfevre,22 Dec 2013 NAK 10R ARIMA KINEN (THE GRAND PRI...,1,K.Ikezoe,57.0,16.0,3.0,6,Gd,2:32.3,-1.3,13-13-12-2,30.3-36.7,36.0,466(0),152.30,Turf,2500
1,Orfevre,6 Oct 2013 5R Prix de l'Arc de Triomphe G1,2,C.Soumillon,59.5,17.0,NaN,6,Sft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Turf,2400
2,Orfevre,15 Sep 2013 6R Prix Foy G2,1,C.Soumillon,58.0,NaN,NaN,7,Sft,2:41.47,NaN,NaN,NaN,NaN,NaN,161.47,Turf,2400
3,Orfevre,31 Mar 2013 HAN 11R SANKEI OSAKA HAI G2,1,K.Ikezoe,58.0,14.0,4.0,5,Gd,1:59.0,-0.1,10-11-9-6,36.7-33.7,33.0,464(+6),119.00,Turf,2000
4,Orfevre,25 Nov 2012 TKY 11R JAPAN CUP G1,2,K.Ikezoe,57.0,17.0,8.0,17,Gd,2:23.1,0.0,12-13-10-3,35.8-34.7,32.9,458(0),143.10,Turf,2400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,Lelouch,22 Apr 2012 TKY 9R ISAWA TOKUBETSU ALW (2 Win),1,H.Shii,57.0,9.0,3.0,3,Gd,1:48.7,-0.1,4-5-4,37.4-34.0,33.5,510(+2),108.70,Turf,1800
495,Lelouch,31 Mar 2012 HAN 12R ４yo+ ALW (2 Win),3,K.Desormeaux,57.0,10.0,7.0,7,Sft,2:07.5,0.3,7-7-5-7,37.7-38.0,37.8,508(+6),127.50,Turf,2000
496,Lelouch,30 Jan 2011 TKY 7R ３yo ALW (1 Win),1,A.Crastus,56.0,10.0,3.0,3,Gd,2:07.2,-0.1,3-5-5,39.0-33.6,33.3,502(+4),127.20,Turf,2000
497,Lelouch,2 Oct 2010 SPR 11R SAPPORO NISAI STAKES G3,4,H.Kitamura,55.0,14.0,4.0,6,Gd,1:50.2,0.4,3-3-3-2,36.1-36.5,36.5,498(+4),110.20,Turf,1800


In [10]:
df1 = df.copy()

In [11]:
df1['Date'] = df1['Race'].str.extract(r'^(\d{1,2}\s+\w+)')     # e.g. "15 Sep"
df1['Year'] = df1['Race'].str.extract(r'\b(\d{4})\b')          # e.g. "2013"
df1['Race Details'] = df1['Race'].str.extract(r'\d{4}\s+(.*)')

In [12]:
df1['Year'] = pd.to_numeric(df1['Year'], errors="coerce")

In [13]:
df1 = df1[df1['Year'] <= 2013]

In [14]:
print(len(df1))

325


In [15]:
df2 = df1[~(
    (df1['Date'].str.strip().str.lower() == '22 dec') &
    (df1['Year'] == 2013)
)]



In [16]:
print(len(df2))

309


In [17]:
df2.head()

,Horse,Race,Finish,Jockey,Weight,Runners,Block,Draw,Dis.,Ground,...,Pos. at Bends,F3F-L3F,L3F,Horse Weight,FT_seconds,Track,Distance,Date,Year,Race Details
1,Orfevre,6 Oct 2013 5R Prix de l'Arc de Triomphe G1,2,C.Soumillon,59.5,17.0,NaN,6,T2400,Sft,...,NaN,NaN,NaN,NaN,NaN,Turf,2400,6 Oct,2013,5R Prix de l'Arc de Triomphe G1
2,Orfevre,15 Sep 2013 6R Prix Foy G2,1,C.Soumillon,58.0,NaN,NaN,7,T2400,Sft,...,NaN,NaN,NaN,NaN,161.47,Turf,2400,15 Sep,2013,6R Prix Foy G2
3,Orfevre,31 Mar 2013 HAN 11R SANKEI OSAKA HAI G2,1,K.Ikezoe,58.0,14.0,4.0,5,T2000,Gd,...,10-11-9-6,36.7-33.7,33.0,464(+6),119.00,Turf,2000,31 Mar,2013,HAN 11R SANKEI OSAKA HAI G2
4,Orfevre,25 Nov 2012 TKY 11R JAPAN CUP G1,2,K.Ikezoe,57.0,17.0,8.0,17,T2400,Gd,...,12-13-10-3,35.8-34.7,32.9,458(0),143.10,Turf,2400,25 Nov,2012,TKY 11R JAPAN CUP G1
5,Orfevre,7 Oct 2012 6R Prix de l'Arc de Triomphe G1,2,C.Soumillon,59.5,18.0,NaN,6,T2400,Sft,...,NaN,NaN,NaN,NaN,NaN,Turf,2400,7 Oct,2012,6R Prix de l'Arc de Triomphe G1


In [50]:
df1.to_excel("/Users/danishluqman/Documents/horse-racing-arima-kinen-2013/data/cleaned_data.xlsx", index=False)

